In [24]:
import os
import requests

r = requests.get(
    url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/notebook_utils.py",
)
open("notebook_utils.py", "w").write(r.text)

r = requests.get(
    url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/pip_helper.py",
)
open("pip_helper.py", "w").write(r.text)

os.environ["GIT_CLONE_PROTECTION_ACTIVE"] = "false"

from pip_helper import pip_install

pip_install(
    "-q",
    "--extra-index-url",
    "https://download.pytorch.org/whl/cpu",
    "llama-index",
    "llama-index-readers-file",
    "llama-index-llms-openvino>=0.2.2",
    "llama-index-embeddings-openvino>=0.2.0",
    "transformers>=4.43.1",
    "openai",
)
pip_install("-q", "git+https://github.com/huggingface/optimum-intel.git", "git+https://github.com/openvinotoolkit/nncf.git", "datasets", "accelerate")
pip_install("--pre", "-Uq", "openvino>=2024.2.0", "--extra-index-url", "https://storage.openvinotoolkit.org/simple/wheels/nightly")
pip_install("--pre", "-Uq", "openvino-tokenizers[transformers]", "--extra-index-url", "https://storage.openvinotoolkit.org/simple/wheels/nightly")

In [57]:
from openai import OpenAI

def generateImageDescriptions(heatmap_url, areas_url):
  client = OpenAI(
      api_key="-",
      base_url="https://fridaplatform.com/v1",
  )

  heatmapDescriptionResponse = client.chat.completions.create(
      model="gpt-4",
      messages=[
          {
          "role": "user",
          "content": [
              {"type": "text", "text": "Describe the heatmap image with extra details"},
              {
              "type": "image_url",
              "image_url": {
                  "url": heatmap_url,
              },
              },
          ],
          }
      ],
      max_tokens=1000,
  )

  diffAreasDescriptionResponse = client.chat.completions.create(
      model="gpt-4",
      messages=[
          {
          "role": "user",
          "content": [
              {"type": "text", "text": "According to the colors, give a detailed description of the areas of the store, how they are divided, what they represent and their position in the store and the image"},
              {
              "type": "image_url",
              "image_url": {
                  "url": areas_url,
              },
              },
          ],
          }
      ],
      max_tokens=1000,
  )

  #Query gpt-4 model for the relation between the heatmap and the area division
  relationsResponse = client.chat.completions.create(
    model="gpt-4",
    messages=[
      {
        "role": "user",
        "content": [
          {"type": "text", "text": "One of this images represents with colors the different areas of a store, and the other one represetns the traffic of the store, red is the liquor, magenta are the sodas and the blue are the newspapers, tell me which areas are the most or less visited"},
          {
            "type": "image_url",
            "image_url": {
              "url": heatmap_url,
            },
          },
          {
            "type": "image_url",
            "image_url": {
              "url": areas_url,
            },
          },
        ],
      }
    ],
    max_tokens=1000,
  )
  open("ragData/descriptions/heatmapDescription.txt", "w+").write(heatmapDescriptionResponse.choices[0].message[0].content[0].text)
  open("ragData/descriptions/diffAreasDescription.txt", "w+").write(diffAreasDescriptionResponse.choices[0].message[0].content[0].text)
  open("ragData/descriptions/relationsDescription.txt", "w+").write(relationsResponse.choices[0].message[0].content[0].text)


ChatCompletionMessage(content='I can’t access or view images on the internet, but I can guide you on how to interpret a heatmap image.', refusal=None, role='assistant', function_call=None, tool_calls=None)


TypeError: 'ChatCompletionMessage' object is not subscriptable

In [15]:
from pathlib import Path
import requests
import io

In [16]:
import ipywidgets as widgets

llm_model_ids = ["OpenVINO/Phi-3-mini-4k-instruct-int4-ov", "meta-llama/Meta-Llama-3.1-8B-Instruct"]

llm_model_id = widgets.Dropdown(
    options=llm_model_ids,
    value=llm_model_ids[0],
    description="Model:",
    disabled=False,
)

llm_model_id

Dropdown(description='Model:', options=('OpenVINO/Phi-3-mini-4k-instruct-int4-ov', 'meta-llama/Meta-Llama-3.1-…

In [17]:
from pathlib import Path
import huggingface_hub as hf_hub

llm_model_path = llm_model_id.value.split("/")[-1]
repo_name = llm_model_id.value.split("/")[0]

if not Path(llm_model_path).exists():
    if repo_name == "OpenVINO":
        hf_hub.snapshot_download(llm_model_id.value, local_dir=llm_model_path)
    else:
        !optimum-cli export openvino --model {llm_model_id.value} --task text-generation-with-past --trust-remote-code --weight-format int4 --group-size 128 --ratio 0.8 {llm_model_path}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

openvino_detokenizer.xml:   0%|          | 0.00/3.25k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.34k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

openvino_tokenizer.bin:   0%|          | 0.00/500k [00:00<?, ?B/s]

In [18]:
embedding_model_id = "BAAI/bge-small-en-v1.5"
embedding_model_path = "bge-small-en-v1.5"

if not Path(embedding_model_path).exists():
    !optimum-cli export openvino --model {embedding_model_id} --task feature-extraction {embedding_model_path}

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2024-09-15 16:21:14.437003: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 16:21:14.468359: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 16:21:14.478674: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-15 16:21:14.505733: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimiz

In [32]:
from notebook_utils import device_widget

llm_device = device_widget("CPU")
llm_device


Dropdown(description='Device:', options=('CPU', 'AUTO'), value='CPU')

In [33]:
from llama_index.llms.openvino import OpenVINOLLM

ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}


def phi_completion_to_prompt(completion):
    return f"<|system|><|end|><|user|>{completion}<|end|><|assistant|>\n"


def llama3_completion_to_prompt(completion):
    return f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{completion}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"


llm = OpenVINOLLM(
    model_id_or_path=str(llm_model_path),
    context_window=3900,
    max_new_tokens=1000,
    model_kwargs={"ov_config": ov_config},
    generate_kwargs={"do_sample": False, "temperature": None, "top_p": None},
    completion_to_prompt=phi_completion_to_prompt if llm_model_path == "Phi-3-mini-4k-instruct-int4-ov" else llama3_completion_to_prompt,
    device_map=llm_device.value,
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPI has co

In [34]:
embedding_device = device_widget()

embedding_device

Dropdown(description='Device:', index=1, options=('CPU', 'AUTO'), value='AUTO')

In [35]:
from llama_index.embeddings.huggingface_openvino import OpenVINOEmbedding

embedding = OpenVINOEmbedding(model_id_or_path=embedding_model_path, device=embedding_device.value)

Compiling the model to AUTO ...
Compiling the model to AUTO ...


In [36]:
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool


def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def divide(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a / b


divide_tool = FunctionTool.from_defaults(fn=divide)

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

def substract(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a - b


substract_tool = FunctionTool.from_defaults(fn=substract)

In [38]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex, Settings

Settings.embed_model = embedding
Settings.llm = llm

reader = SimpleDirectoryReader("./ragData", recursive=True)
documents = reader.load_data()
index = VectorStoreIndex.from_documents(
    documents,
)

In [39]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    index.as_query_engine(streaming=True),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for basic facts about the documents",
    ),
)

In [40]:
agent = ReActAgent.from_tools([multiply_tool, divide_tool, add_tool, substract_tool, vector_tool], llm=llm, verbose=True)

In [74]:
question = input("Ask me a question: ")
while(question != "quit"):
    response = agent.chat(question)
    print(str(response))
    question = input("Ask me a question: ")

Ask me a question: quit


In [64]:
def query_agent(query):
    return str(agent.chat(query))

In [51]:
!pip install websockets
!pip install boto3
!pip install filterpy
!pip install ultralytics
!pip install uvicorn
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 4.8 MB/s eta 0:00:00


In [65]:
import asyncio
import websockets
import json
import boto3
from datetime import datetime
from botocore.client import Config
from computerVision import apply_model_to_video

# CLOUDFLARE CONFIG
ACCESS_KEY_ID = 'b19d514f16f18783ff2a290c04f619be'
SECRET_ACCESS_KEY = 'bd6a46cbdf3fd63f718caba171cd8fd5ecd917db709246499dc60371b935d31d'
BUCKET_NAME = 'hackmty'
REGION_NAME = 'auto'  # Use 'auto' as the region name for Cloudflare
ENDPOINT_URL = 'https://6619572a94d5a1d039dc239d940199ed.r2.cloudflarestorage.com'

async def getHeatmaps(websocket, path):
    # <LOGICA DE OBTENCION DE HEATMAPS>

    # <CLOUDFLARE>
    session = boto3.session.Session()
    processedHeatmaps = False

    s3_client = session.client(
        's3',
        region_name=REGION_NAME,
        endpoint_url=ENDPOINT_URL,
        aws_access_key_id=ACCESS_KEY_ID,
        aws_secret_access_key=SECRET_ACCESS_KEY,
        config=Config(signature_version='s3v4')  # R2 uses S3v4 signature
    )

    # Function to upload the file and get its public URL
    def upload_file_to_r2(file_path, object_name=None):
        if object_name is None:
            object_name = file_path

        try:
            # Upload the file
            s3_client.upload_file(file_path, BUCKET_NAME, object_name)

            # Generate the public URL (assuming the file is publicly accessible)
            public_url = f"{ENDPOINT_URL}/{BUCKET_NAME}/{object_name}"

            return public_url

        except Exception as e:
            print(f"Error uploading file: {e}")
            return None

    async for message in websocket:
        if (not processedHeatmaps):
            processedHeatmaps = True
            url = message

            video_output_path = 'video_output.avi'
            heatmap_path = 'heatmap.jpg'
            last_frame_path = 'last_frame.jpg'

            apply_model_to_video(url, video_output_path, heatmap_path, last_frame_path)

            video_url = upload_file_to_r2(video_output_path, "video" + datetime.now().strftime("%Y%m%d%H%M%S") + ".avi")
            heatmap_url = upload_file_to_r2(heatmap_path, "heatmap" + datetime.now().strftime("%Y%m%d%H%M%S") + ".jpg")
            last_frame_url = upload_file_to_r2(last_frame_path, "last_frame" + datetime.now().strftime("%Y%m%d%H%M%S") + ".jpg")
            file_urls = [video_url, last_frame_url]
            generateImageDescriptions(heatmap_url, last_frame_url)

            await websocket.send(json.dumps(file_urls))
        else:
            agent_response = await query_agent(query)

            # Send agent response back to the client
            await websocket.send(json.dumps(agent_response))


async def echo(websocket, path):
    if path == '/heatmaps':
        await getHeatmaps(websocket, path)
    else:
        await websocket.send("Unknown path")

async def runApiLLM():
    async with websockets.serve(echo, "localhost", 8765):
        print("WebSocket server running on ws://localhost:8765")
        await asyncio.Future()

In [73]:
result = await runApiLLM()

OSError: [Errno 99] error while attempting to bind on address ('::1', 8765, 0, 0): cannot assign requested address